# LDA Topic Model

Xavier C. Gill
September 16, 2022

## Library Imports

In [2]:
import gensim
import nltk
nltk.download('stopwords')
import numpy as np
import json
import glob
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.gensim_models as gensimviz
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()
from concurrent.futures import ThreadPoolExecutor
import openpyxl
import sklearn

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xaviergill/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/Users/xaviergill/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## Retrieving Texts

Reading the scraped csv file as our input. This file contains all relevant fields exploded by author.

In [3]:
df = pd.read_csv('../data/test/scraped.csv')
df = df.drop(columns='Unnamed: 0')
df

,aem_path,page_title,template,last_modified,tags,topic,program,theme,content_type_tags,user_role,...,person,team,last_modified_by,created_by,replication_status,authors,user_role_param,status_code,url_with_params,publish_date
0,/content/invesco/us/en/insights/election-2020-...,Election 2020: The Senate is definitely in play,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 3 Nov 2020 15:06:30 +0000",['nan'],"['Legislative and regulatory', 'Washington upd...",['Election 2020'],['Elections'],Insight,['Institutional'],...,NaN,NaN,katta.dheeraj,suyog.deshmukh,NOT_ACTIVATED,andy-blocker,['Institutional'],200,https://www.invesco.com/us/en/insights/electio...,2020-10-30
1,/content/invesco/us/en/insights/election-2020-...,Election 2020: The Senate is definitely in play,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 3 Nov 2020 15:06:30 +0000",['nan'],"['Legislative and regulatory', 'Washington upd...",['Election 2020'],['Elections'],Insight,['Institutional'],...,NaN,NaN,katta.dheeraj,suyog.deshmukh,NOT_ACTIVATED,jennifer-flitton,['Institutional'],200,https://www.invesco.com/us/en/insights/electio...,2020-10-30
2,/content/invesco/us/en/insights/tactical-asset...,Tactical asset allocation outlook: November 2020,/conf/invesco/settings/wcm/templates/article-p...,"Wed, 25 Nov 2020 15:12:43 +0000",['nan'],"['Multi Asset', 'Asset allocation', 'Markets a...",['Solutions'],['Investment solutions'],Insight,['Institutional'],...,NaN,NaN,priya.sasidharan,priya.sasidharan,NOT_ACTIVATED,alessio-de-longis,['Institutional'],200,https://www.invesco.com/us/en/insights/tactica...,2020-11-11
3,/content/invesco/us/en/insights/2021-outlook-i...,2021 Investment Outlook: Implications for worl...,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 1 Dec 2020 10:05:35 +0000",['nan'],['Market outlook'],['nan'],['Macroeconomics & outlook'],Insight,['Institutional'],...,NaN,NaN,priya.sasidharan,priya.sasidharan,NOT_ACTIVATED,invesco-global-market-strategy-office,['Institutional'],200,https://www.invesco.com/us/en/insights/2021-ou...,2020-11-30
4,/content/invesco/us/en/insights/shifting-dc-ti...,Shifting DC Times: Helping women close the ret...,/conf/invesco/settings/wcm/templates/article-p...,"Mon, 14 Dec 2020 16:46:55 +0000",['nan'],"['Participant engagement', 'Defined contributi...",['Shifting DC Times'],['nan'],Insight,"['Financial Professional', 'Institutional']",...,NaN,NaN,suyog.deshmukh,nick.miller,NOT_ACTIVATED,invesco,"['FinancialProfessional', 'Institutional']",200,https://www.invesco.com/us/en/insights/shiftin...,2019-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,/content/invesco/us/en/insights/liability-awar...,Liability Aware Investing: A more holistic app...,/conf/invesco/settings/wcm/templates/article-p...,"Mon, 22 Aug 2022 17:53:42 +0000",['nan'],['Portfolio Construction'],['nan'],['nan'],Insight,['Institutional'],...,NaN,NaN,saikumar.ambadi,saikumar.ambadi,NOT_ACTIVATED,paisley-nardini,['Institutional'],200,https://www.invesco.com/us/en/insights/liabili...,2022-08-19
809,/content/invesco/us/en/insights/new-word-order,"""New word order"" program overview",/conf/invesco/settings/wcm/templates/article-p...,"Mon, 22 Aug 2022 18:37:18 +0000",['nan'],['Client Service'],['nan'],['Your practice'],Insight,['Financial Professional'],...,NaN,NaN,bharadwaj.cheruvu,neha.arora,NOT_ACTIVATED,invesco-global-consulting,['FinancialProfessional'],200,https://www.invesco.com/us/en/insights/new-wor...,NaN
810,/content/invesco/us/en/insights/what-matters-m...,"""Energy management for high performance"" progr...",/conf/invesco/settings/wcm/templates/article-p...,"Mon, 22 Aug 2022 18:40:30 +0000",['nan'],"['Client Service', 'Practice Management']",['Roadmap to Recovery'],['Your practice'],Insight,['Financial Professional'],...,NaN,NaN,bharadwaj.cheruvu,neha.arora,NOT_ACTIVATED,invesco-global-consulting,['FinancialProfessional'],200,https://www.invesco.com/us/en/insights/what-ma...,NaN
811,/content/inve

In [4]:
# Use thread pool executor to perform IO operations
# url_list = df['url_with_params'].tolist()
#
# def get_content(url: str):
#     response = requests.get(url)
#     content = response.content.decode('utf-8')
#     return content
#
# with tqdm(total=len(url_list)) as pbar:
#     with ThreadPoolExecutor() as executor:
#         results = executor.map(get_content, url_list)

In [5]:
def retrieve_content(row):
    url = row['url_with_params']
    response = requests.get(url)
    row['content'] = response.content.decode('utf-8')
    return row

In [6]:
df = df.progress_apply(retrieve_content, axis=1)

100%|██████████| 813/813 [10:58<00:00,  1.23it/s]


In [7]:
df.head()

,aem_path,page_title,template,last_modified,tags,topic,program,theme,content_type_tags,user_role,...,team,last_modified_by,created_by,replication_status,authors,user_role_param,status_code,url_with_params,publish_date,content
0,/content/invesco/us/en/insights/election-2020-...,Election 2020: The Senate is definitely in play,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 3 Nov 2020 15:06:30 +0000",['nan'],"['Legislative and regulatory', 'Washington upd...",['Election 2020'],['Elections'],Insight,['Institutional'],...,NaN,katta.dheeraj,suyog.deshmukh,NOT_ACTIVATED,andy-blocker,['Institutional'],200,https://www.invesco.com/us/en/insights/electio...,2020-10-30,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ..."
1,/content/invesco/us/en/insights/election-2020-...,Election 2020: The Senate is definitely in play,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 3 Nov 2020 15:06:30 +0000",['nan'],"['Legislative and regulatory', 'Washington upd...",['Election 2020'],['Elections'],Insight,['Institutional'],...,NaN,katta.dheeraj,suyog.deshmukh,NOT_ACTIVATED,jennifer-flitton,['Institutional'],200,https://www.invesco.com/us/en/insights/electio...,2020-10-30,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ..."
2,/content/invesco/us/en/insights/tactical-asset...,Tactical asset allocation outlook: November 2020,/conf/invesco/settings/wcm/templates/article-p...,"Wed, 25 Nov 2020 15:12:43 +0000",['nan'],"['Multi Asset', 'Asset allocation', 'Markets a...",['Solutions'],['Investment solutions'],Insight,['Institutional'],...,NaN,priya.sasidharan,priya.sasidharan,NOT_ACTIVATED,alessio-de-longis,['Institutional'],200,https://www.invesco.com/us/en/insights/tactica...,2020-11-11,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ..."
3,/content/invesco/us/en/insights/2021-outlook-i...,2021 Investment Outlook: Implications for worl...,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 1 Dec 2020 10:05:35 +0000",['nan'],['Market outlook'],['nan'],['Macroeconomics & outlook'],Insight,['Institutional'],...,NaN,priya.sasidharan,priya.sasidharan,NOT_ACTIVATED,invesco-global-market-strategy-office,['Institutional'],200,https://www.invesco.com/us/en/insights/2021-ou...,2020-11-30,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ..."
4,/content/invesco/us/en/insights/shifting-dc-ti...,Shifting DC Times: Helping women close the ret...,/conf/invesco/settings/wcm/templates/article-p...,"Mon, 14 Dec 2020 16:46:55 +0000",['nan'],"['Participant engagement', 'Defined contributi...",['Shifting DC Times'],['nan'],Insight,"['Financial Professional', 'Institutional']",...,NaN,suyog.deshmukh,nick.miller,NOT_ACTIVATED,invesco,"['FinancialProfessional', 'Institutional']",200,https://www.invesco.com/us/en/insights/shiftin...,2019-06-01,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ..."


In [8]:
# Parse html content for relevant tags
content_list = df['content'].to_list()

def parse_text(html_content: str) -> str:
    text_parts = list()
    soup = BeautifulSoup(html_content, parser='lxml')
    key_takeaway_tag = soup.find('div', class_='key-takeaways__list')
    if key_takeaway_tag:
        # Get text of all children
        key_takeaways = key_takeaway_tag.get_text()
        # Append takeaways to list to be joined with body text later
        text_parts.append(key_takeaways)
    body_text_tag = soup.find('section', class_='rich-text-editor')
    if body_text_tag:
        body_text = body_text_tag.get_text()
        text_parts.append(body_text)
    # Join text parts with return separator
    text = '\n'.join(text_parts)

    return text

def extract_texts(html_content: list, function: object) -> list:
    text_list = list()
    with tqdm(total=len(html_content)) as pbar:
        with ThreadPoolExecutor() as executor:
            results = executor.map(function, html_content)
            for result in results:
                text_list.append(result)
                pbar.update(1)

    return text_list

Running the multithreaded text extraction function on the list of all html content retrieved with the requests library.

In [9]:
# Run functions
texts = extract_texts(content_list, parse_text)

100%|██████████| 813/813 [00:49<00:00, 16.45it/s] 


Add text list to the datafame.

In [10]:
df['texts'] = texts
df.head()

,aem_path,page_title,template,last_modified,tags,topic,program,theme,content_type_tags,user_role,...,last_modified_by,created_by,replication_status,authors,user_role_param,status_code,url_with_params,publish_date,content,texts
0,/content/invesco/us/en/insights/election-2020-...,Election 2020: The Senate is definitely in play,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 3 Nov 2020 15:06:30 +0000",['nan'],"['Legislative and regulatory', 'Washington upd...",['Election 2020'],['Elections'],Insight,['Institutional'],...,katta.dheeraj,suyog.deshmukh,NOT_ACTIVATED,andy-blocker,['Institutional'],200,https://www.invesco.com/us/en/insights/electio...,2020-10-30,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ...",\n\n\nRepublicans on the defensive\n\n1\nBiden...
1,/content/invesco/us/en/insights/election-2020-...,Election 2020: The Senate is definitely in play,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 3 Nov 2020 15:06:30 +0000",['nan'],"['Legislative and regulatory', 'Washington upd...",['Election 2020'],['Elections'],Insight,['Institutional'],...,katta.dheeraj,suyog.deshmukh,NOT_ACTIVATED,jennifer-flitton,['Institutional'],200,https://www.invesco.com/us/en/insights/electio...,2020-10-30,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ...",\n\n\nRepublicans on the defensive\n\n1\nBiden...
2,/content/invesco/us/en/insights/tactical-asset...,Tactical asset allocation outlook: November 2020,/conf/invesco/settings/wcm/templates/article-p...,"Wed, 25 Nov 2020 15:12:43 +0000",['nan'],"['Multi Asset', 'Asset allocation', 'Markets a...",['Solutions'],['Investment solutions'],Insight,['Institutional'],...,priya.sasidharan,priya.sasidharan,NOT_ACTIVATED,alessio-de-longis,['Institutional'],200,https://www.invesco.com/us/en/insights/tactica...,2020-11-11,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ...",\n\nThe return of the coronavirus across Europ...
3,/content/invesco/us/en/insights/2021-outlook-i...,2021 Investment Outlook: Implications for worl...,/conf/invesco/settings/wcm/templates/article-p...,"Tue, 1 Dec 2020 10:05:35 +0000",['nan'],['Market outlook'],['nan'],['Macroeconomics & outlook'],Insight,['Institutional'],...,priya.sasidharan,priya.sasidharan,NOT_ACTIVATED,invesco-global-market-strategy-office,['Institutional'],200,https://www.invesco.com/us/en/insights/2021-ou...,2020-11-30,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ...","\n\nOur base, upside, and downside scenarios f..."
4,/content/invesco/us/en/insights/shifting-dc-ti...,Shifting DC Times: Helping women close the ret...,/conf/invesco/settings/wcm/templates/article-p...,"Mon, 14 Dec 2020 16:46:55 +0000",['nan'],"['Participant engagement', 'Defined contributi...",['Shifting DC Times'],['nan'],Insight,"['Financial Professional', 'Institutional']",...,suyog.deshmukh,nick.miller,NOT_ACTIVATED,invesco,"['FinancialProfessional', 'Institutional']",200,https://www.invesco.com/us/en/insights/shiftin...,2019-06-01,"\n<!DOCTYPE HTML>\n<html lang=""en-us"">\n\n ...",\n\nThe retirement savings gap between men and...


## Preprocess Texts

Retrieving stopwords

In [11]:
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Lemmatizing texts using spacy.

In [12]:
def lemmatize(data, allowed_postags=None) -> list[str]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    texts_out = []
    with tqdm(total=len(data)) as pbar:
        for text in data:
            doc = nlp(text)
            new_text = []
            for token in doc:
                if token.pos_ in allowed_postags:
                    new_text.append(token.lemma_)
            joined = ' '.join(new_text)
            texts_out.append(joined)
            pbar.update(1)

    return texts_out

In [13]:
# Running lemmatization function on data
lemmatized_texts = lemmatize(texts)

100%|██████████| 813/813 [00:39<00:00, 20.76it/s] 


In [14]:
print(lemmatized_texts[0][:100])

defensive lead cash support democratic candidate back foot red state good bad news democrat more rep


Preprocessing lemmetized texts with gensim simple preprocess.

In [15]:
def generate_words(data: list) -> list:
    final = []
    for text in tqdm(data):
        new_text = simple_preprocess(text, deacc=True)
        final.append(new_text)

    return final

In [16]:
# Generating words with generator function
text_words = generate_words(lemmatized_texts)

100%|██████████| 813/813 [00:00<00:00, 1468.88it/s]


In [17]:
print(text_words[0][:100])

['defensive', 'lead', 'cash', 'support', 'democratic', 'candidate', 'back', 'foot', 'red', 'state', 'good', 'bad', 'news', 'democrat', 'more', 'republican', 'seat', 'play', 'many', 'usually', 'reliable', 'red', 'state', 'several', 'scenario', 'consider', 'regardless', 'win', 'presidency', 'fortune', 'next', 'administration', 'rest', 'solely', 'hand', 'year', 'ago', 'look', 'map', 'seat', 'conclude', 'difficult', 'path', 'back', 'majority', 'fast', 'forward', 'year', 'persistent', 'lead', 'former', 'gusher', 'campaign', 'cash', 'democratic', 'candidate', 'suddenly', 'play', 'defense', 'usually', 'reliable', 'red', 'state', 'good', 'news', 'more', 'republican', 'seat', 'play', 'democratic', 'seat', 'bad', 'news', 'many', 'seat', 'red', 'state', 'nation', 'just', 'day', 'away', 'election', 'consider', 'factor', 'give', 'democratic', 'challenger', 'hope', 'republican', 'incumbent', 'cause', 'concern', 'include', 'money', 'long', 'say', 'money', 'mother', 'milk', 'politic']


Creating a corpus.

It is important to note that our corpus is relatively small, thus should likely be expanded. It could also be pertinent to expend the corpus outside of our scope by adding 3rd party finance specific dictionaries.

In [18]:
id2word = corpora.Dictionary(text_words)
corpus = []
for text in tqdm(text_words):
    bag_of_words = id2word.doc2bow(text)
    corpus.append(bag_of_words)

100%|██████████| 813/813 [00:00<00:00, 5903.11it/s]


In [19]:
print(corpus[0][0:20])

[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 3), (18, 1), (19, 1)]


Creating the LDA model.

In [53]:
model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,
    num_topics=12,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto'
)

## Visualizing the result

In [54]:
pyLDAvis.enable_notebook()
vis = gensimviz.prepare(model, corpus, id2word, mds='mmds', R=10)

/Users/xaviergill/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [55]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.303373 -0.030370       1        1  27.061407
2      0.159012 -0.307878       2        1  22.797132
7     -0.213430 -0.264225       3        1  15.484968
11    -0.051833 -0.324642       4        1  13.965277
6      0.316221 -0.198429       5        1   5.556776
3     -0.333213 -0.035675       6        1   4.487557
5      0.238405  0.212091       7        1   3.115188
8     -0.091564 -0.040354       8        1   2.884497
9      0.088689  0.278266       9        1   2.436400
4     -0.145112  0.406534      10        1   1.234045
1     -0.252909  0.182258      11        1   0.972670
10    -0.017638  0.122424      12        1   0.004083, topic_info=          Term         Freq        Total Category  logprob  loglift
173     market  1583.000000  1583.000000  Default  10.0000  10.0000
320   investor   653.000000   653.000000  Default   9.0000   9.0000
163     growth   793.000000   793.000000  Default   8.0000   8.0000
159  financial   424.000000   424.000000  Default   7.0000   7.0000
336       plan   466.000000   466.000000  Default   6.0000   6.0000
..         ...          ...          ...      ...      ...      ...
5        allow     0.000823   102.124458  Topic12  -8.8317  -1.6225
6    ambitious     0.000823     8.701106  Topic12  -8.8317   0.8403
7       amount     0.000823    55.189246  Topic12  -8.8317  -1.0071
8           as     0.000823   340.970289  Topic12  -8.8317  -2.8281
9         away     0.000823    41.652948  Topic12  -8.8317  -0.7257

[266 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.345966    about
0         3  0.155685    about
0         4  0.501650    about
658       1  0.089464  account
658       4  0.119285  account
...     ...       ...      ...
132       6  0.113190     year
132       7  0.015033     year
132       8  0.040678     year
777       1  0.852726    yield
777       6  0.142596    yield

[552 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 8, 12, 7, 4, 6, 9, 10, 5, 2, 11])